In [1]:
import numpy as np
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
from torch.optim import SGD
from torch.nn import CrossEntropyLoss, MSELoss
from pennylane import NesterovMomentumOptimizer
from thesis.qcnn import QCNN

dims = (16, 16), (28, 28), (32, 32)
num_trials = 1

In [2]:
from thesis.operation.ansatz import BaselineAnsatz
from thesis.fn.machine_learning import DatasetOptions
from thesis.fn.machine_learning import image_transform

qcnn = QCNN((16, 16))
qcnn.dataset_options = DatasetOptions(image_transform(qcnn.dims), None, classes=[0, 1], batch_size=40)
qcnn.optimizer_options = {"lr": 0.01, "momentum": 0.9, "nesterov": True}

for _ in range(num_trials):
    accuracy = qcnn(BaselineAnsatz, FashionMNIST, SGD, CrossEntropyLoss())
    print(f"New ansatz: {accuracy=:.3%}")

tensor([ 0.5391, -0.2533, -0.3848,  1.4358, -0.2188, -0.1062, -1.9151, -1.6908,
        -0.3419,  0.6648, -0.5320, -0.7972, -0.2216,  1.7903, -1.2212],
       grad_fn=<TransposeBackward0>) <Wires = [0, 7]>
tensor([ 0.5391, -0.2533, -0.3848,  1.4358, -0.2188, -0.1062, -1.9151, -1.6908,
        -0.3419,  0.6648, -0.5320, -0.7972, -0.2216,  1.7903, -1.2212],
       grad_fn=<TransposeBackward0>) <Wires = [0, 1]>
tensor([ 0.5391, -0.2533, -0.3848,  1.4358, -0.2188, -0.1062, -1.9151, -1.6908,
        -0.3419,  0.6648, -0.5320, -0.7972, -0.2216,  1.7903, -1.2212],
       grad_fn=<TransposeBackward0>) <Wires = [2, 3]>
tensor([ 0.5391, -0.2533, -0.3848,  1.4358, -0.2188, -0.1062, -1.9151, -1.6908,
        -0.3419,  0.6648, -0.5320, -0.7972, -0.2216,  1.7903, -1.2212],
       grad_fn=<TransposeBackward0>) <Wires = [4, 5]>
tensor([ 0.5391, -0.2533, -0.3848,  1.4358, -0.2188, -0.1062, -1.9151, -1.6908,
        -0.3419,  0.6648, -0.5320, -0.7972, -0.2216,  1.7903, -1.2212],
       grad_fn=<Transpos